In [1]:
from IPython.display import clear_output
from PIL import Image
import os
import json
import pandas as pd

def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% Compressed'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

plays = pd.read_csv("nfl-big-data-bowl-2022/plays.csv")
plays = downcast(plays)

43.0% Compressed


In [10]:
plays['gameClock'] = pd.to_timedelta(plays['gameClock'])
plays['gameClock']
pd.set_option('display.max_columns', None)
plays.head()
plays.dtypes

gameId                              int32
playId                              int16
playDescription                    object
quarter                              int8
down                                 int8
yardsToGo                            int8
possessionTeam                     object
specialTeamsPlayType               object
specialTeamsResult                 object
kickerId                          float32
returnerId                         object
kickBlockerId                     float32
yardlineSide                       object
yardlineNumber                       int8
gameClock                 timedelta64[ns]
penaltyCodes                       object
penaltyJerseyNumbers               object
penaltyYards                      float32
preSnapHomeScore                     int8
preSnapVisitorScore                  int8
passResult                         object
kickLength                        float32
kickReturnYardage                 float32
playResult                        

In [18]:
# plays['returnerId'] = plays['returnerId'].loc[plays['']]

42436.0    3
43387.0    2
46148.0    2
45533.0    2
47889.0    2
          ..
52565.0    1
52592.0    1
42394.0    1
46633.0    1
45063.0    1
Name: kickBlockerId, Length: 83, dtype: int64

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

In [11]:
df = plays.loc[plays['specialTeamsPlayType']=='Kickoff'].drop('specialTeamsPlayType',axis=1)

In [23]:
from sklearn.model_selection import train_test_split

target = df['playResult']
data = df.drop(['playResult','playDescription','possessionTeam','specialTeamsResult','returnerId','yardlineSide','gameClock','penaltyCodes','penaltyJerseyNumbers','passResult'],axis=1)
data = data.drop(['kickerId','kickBlockerId','kickLength','kickReturnYardage'],axis=1)
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.25, random_state=0)

In [20]:
from sklearn.linear_model import LogisticRegression

LogReg = LogisticRegression()
LogReg.fit(x_train, y_train)
predictions = LogReg.predict(x_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
score = LogReg.score(x_test, y_test)
print(score)